In [1]:
from funcs.fix_continuous_responses import *

In [2]:
participant = 'moiern'

### Load log data

In [3]:
# define column names
col_names = ['timestamp', 'entry_type', 'message']

# get the path of the file with '.log' extension
log_file_path = glob.glob(os.path.join(get_path('raw_data'), participant, 'Continuous', '*.log'))[-1]

# read the log file into a DataFrame
df_log = pd.read_csv(log_file_path, sep='\t', names=col_names)
# Remove leading and trailing whitespaces
df_log = df_log.applymap(lambda x: x.strip() if isinstance(x, str) else x)

### Parse log

In [4]:
df_trials_log = parse_log(df_log)

# print the new DataFrame
df_trials_log.head()

,index,log_timestamp,rep_number,log_responses,log_rt
0,0,69.7350,2,1,[2.491500000000002]
1,1,72.4483,3,1,[2.333199999999991]
2,2,75.0061,4,1,[2.551099999999991]
3,3,77.7832,5,1,[1.4614000000000118]
4,4,79.4768,6,1,[2.9133000000000067]


### Load matching csv file

In [5]:
# read the matching csv file into a DataFrame
df_csv = pd.read_csv(log_file_path[:-4] + "_1.csv")

df_csv.rename(columns={'feedback.keys': 'responses',
                       'Prediction': 'pred'}, inplace=True)

# fix feedback.rt format
df_csv = fix_feedbackRT_format(df_csv)
# df_csv['feedback.rt'] = df_csv['feedback.rt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# show the first few rows of the DataFrame
df_csv.head()

,sweeps.thisRepN,sweeps.thisTrialN,sweeps.thisN,sweeps.thisIndex,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,instructions.started,instructions.stopped,...,end_txt.stopped,participant,session,language (english or french),Resume previous experiment,date,expName,psychopyVersion,frameRate,Unnamed: 50
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.644416,None,...,NaN,moiern,1.0,fr,1.0,2022-07-08_14h28.56.466,Continuous,2022.1.1,35.959581,NaN
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,moiern,1.0,fr,1.0,2022-07-08_14h28.56.466,Continuous,2022.1.1,35.959581,NaN
2,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,NaN,NaN,...,NaN,moiern,1.0,fr,1.0,2022-07-08_14h28.56.466,Continuous,2022.1.1,35.959581,NaN
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,NaN,NaN,...,NaN,moiern,1.0,fr,1.0,2022-07-08_14h28.56.466,Continuous,2022.1.1,35.959581,NaN
4,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,NaN,NaN,...,NaN,moiern,1.0,fr,1.0,2022-07-08_14h28.56.466,Continuous,2022.1.1,35.959581,NaN


In [6]:
df_trials_csv = correct_continuous_responses(df_csv)

df_trials_csv = df_trials_csv.loc[(df_trials_csv['rep_number'] > 1)]  # for keeping R:  | (df_trials_csv['pred'] == 'none')]
df_trials_csv['index'] = np.linspace(0, len(df_trials_csv) - 1, len(df_trials_csv))
# df_trials_csv['feedback.rt'] = df_trials_csv['feedback.rt'].astype(float)

# show the first few rows of the new DataFrame
df_trials_csv.head()

Filtering RTs < 1sec after the tone onset: 12 detections removed


,feedback.started,rep_number,responses,feedback.rt,pred,corrected_rts,corrected_responses,index
2,69.763261,2.0,['space'],[-2.7036324000218883],none,[70.17807789996732],1.0,0.0
3,72.473964,3.0,['space'],[-2.2958861000370234],none,[72.8504655000288],1.0,1.0
4,75.038301,4.0,['space'],[-2.1878352999920025],none,[75.41059270000551],1.0,2.0
5,77.808075,5.0,['space'],[-2.3974821999436244],none,[78.21114979998674],1.0,3.0
6,79.498505,6.0,['space'],[-1.2873554000398144],none,[79.89844359993003],1.0,4.0


### Check against log responses

In [7]:
merged_df = pd.merge(df_trials_csv, df_trials_log, on='index')

print(f"Of the {len(merged_df)} trials considered:")

num_diff_rows = (merged_df['feedback.rt'].apply(lambda x: 1 if len(x) > 0 else 0) != merged_df['log_responses']).sum()
print(f"\n- Before shifting answers: {num_diff_rows} differences between .log and .csv responses")


num_diff_rows = (merged_df['corrected_rts'].apply(lambda x: 1 if len(x) > 0 else 0) != merged_df['log_responses']).sum()
print(f"\n- After shifting answers: {num_diff_rows} differences between .log and .csv responses")


num_diff_rows = (merged_df['corrected_responses'] != merged_df['log_responses']).sum()
print(f"\n- After filtering RTs < 1 sec post tone onset: {num_diff_rows} differences between .log and .csv responses")

Of the 213 trials considered:

- Before shifting answers: 55 differences between .log and .csv responses

- After shifting answers: 16 differences between .log and .csv responses

- After filtering RTs < 1 sec post tone onset: 78 differences between .log and .csv responses
